In [ ]:
print('------------Training Xgboost------------')
%run ./Preprocessing.ipynb

In [1]:
import optuna
from sklearn.model_selection import (
    StratifiedKFold, KFold
)
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost.callback import EvaluationMonitor
from xgboost.callback import EarlyStopping
import plotly

c:\dev\.conda\envs\jupyter-lab\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
opt_grid = {
        'early_stopping': 5,
        'number_of_trials': 5,
        'shuffle': True,
        'cv': KFold(n_splits=10, random_state=None, shuffle=True),
        'time_constraint': 60 * 1
    }
    
static_param = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc'
}

In [5]:
def objective(trial): 
    
    param_grid = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.8),
        # "num_leaves": trial.suggest_int("num_leaves", 31, 63, 5),
        "max_depth": trial.suggest_int("max_depth", 2, 30),
        'min_child_weight': trial.suggest_float("min_child_weight", 1, 500),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1),
        "colsample_bylevel": trial.suggest_float("colsample_bytree", 0.1, 1),
        "subsample": trial.suggest_float("subsample", 0.1, 1),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1, 5)
    }

    full_grid = {**param_grid, **static_param}

    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "test-" + full_grid['eval_metric'])
    early_callback = xgb.callback.EarlyStopping(5)


    cv_results = xgb.cv(full_grid,
                                dtrain=xgb.DMatrix(xtrain, label=ytrain),
                                folds= opt_grid['cv'],
                                metrics=full_grid['eval_metric'],
                                callbacks=[pruning_callback, early_callback]
                                )

    prep_string = 'test-' + full_grid['eval_metric'] + '-mean'
    scores = np.mean(cv_results[prep_string])


    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    return scores

study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize')
study.optimize(objective,n_trials=opt_grid['number_of_trials'],show_progress_bar=False,timeout=opt_grid['time_constraint'])
model_parameters = study.best_params
fig = optuna.visualization.plot_intermediate_values(study)
fig.show()

[I 2022-07-31 17:12:59,245] A new study created in memory with name: no-name-16451767-3699-4727-a1e8-5bb58dacde7a
[I 2022-07-31 17:13:02,364] Trial 0 finished with value: 0.7623449800000001 and parameters: {'learning_rate': 0.11618405884014912, 'max_depth': 22, 'min_child_weight': 298.9901248266855, 'colsample_bytree': 0.3406618318329484, 'subsample': 0.766845025132358, 'reg_alpha': 1.517908134342985, 'reg_lambda': 2.925534557146501}. Best is trial 0 with value: 0.7623449800000001.
[I 2022-07-31 17:13:04,888] Trial 1 finished with value: 0.79658181 and parameters: {'learning_rate': 0.33019397865802436, 'max_depth': 28, 'min_child_weight': 263.27719218878514, 'colsample_bytree': 0.4718283802434422, 'subsample': 0.1310975040686739, 'reg_alpha': 3.8857244398843966, 'reg_lambda': 3.3475222662387063}. Best is trial 1 with value: 0.79658181.
[I 2022-07-31 17:13:08,195] Trial 2 finished with value: 0.8285322900000001 and parameters: {'learning_rate': 0.4906587906755794, 'max_depth': 22, 'min_

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [21]:
model_xgb = XGBClassifier(use_label_encoder= False)

In [22]:
model_xgb.set_params(**model_parameters)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8550977691995645,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.49777240282595053, max_delta_step=None,
              max_depth=19, min_child_weight=259.08880877072255, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=4.200286096725215, reg_lambda=2.697486178623413,
              scale_pos_weight=None, subsample=0.6031914948953285,
              tree_method=None, use_label_encoder=False,
              validate_parameters=None, verbosity=None)

In [23]:
model_xgb=model_xgb.fit(xtrain,ytrain)
pred_xgb=model_xgb.predict(xtest)
probs_xgb = model_xgb.predict_proba(xtest)[:, 1]

print('-----------model_xgb, pred_xgb, probs_xgb loaded-----------')

[15:30:10] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
-----------model_xgb, pred_xgb, probs_xgb loaded-----------
